In [ ]:
import os

from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives.asymmetric import rsa

In [ ]:
default_key_path = "~/.ssh/magic_packet.pem"
default_key_name = "magic_packet"
default_region = "us-east-1"  # TODO: use aws sdk to retrieve this from config

# RSA key pair

The key pair is used to SSH into the EC2 instance. For more on Amazon EC2 key pairs see [Amazon EC2 key pairs and Linux instances](https://docs.aws.amazon.com/AWSEC2/latest/UserGuide/ec2-key-pairs.html).

In [ ]:
def read_public_key(private_key_path):
    key_path = os.path.expanduser(private_key_path)
    with open(key_path, "rb") as key_file:
        private_key = serialization.load_pem_private_key(key_file.read(), password=None)
        public_key = private_key.public_key()
        public_bytes = public_key.public_bytes(
            serialization.Encoding.OpenSSH, serialization.PublicFormat.OpenSSH
        )
        return public_bytes.decode("utf-8")

In [ ]:
def write_private_key(private_key_path, passphrase):
    private_key = rsa.generate_private_key(
        public_exponent=65537,
        key_size=2048,
    )
    key_path = os.path.expanduser(private_key_path)
    encryption_algorithm = (
        serialization.BestAvailableEncryption(passphrase)
        if passphrase
        else serialization.NoEncryption()
    )
    with open(key_path, "wb") as key_file:
        key_file.write(
            private_key.private_bytes(
                encoding=serialization.Encoding.PEM,
                format=serialization.PrivateFormat.TraditionalOpenSSL,
                encryption_algorithm=encryption_algorithm,
            )
        )

## (Optional) Create an RSA key pair

In [ ]:
key_path = (
    input(f"Enter the RSA private key path (or {default_key_path}): ")
    or default_key_path
)
write_private_key(
    key_path,
    input('Enter the passhprase (or ""): '),
)

Make sure to set the proper permissions on the key file ([Connect to your Linux instance using SSH](https://docs.aws.amazon.com/AWSEC2/latest/UserGuide/AccessingInstancesLinux.html))

In [ ]:
!chmod 400 $key_path

# Specify terraform input variables

In [ ]:
# These variables are specified in the *.tf files
variables = {
    "public_key": "",
    "key_name": default_key_name,
    "region": default_region,
}

In [ ]:
# Retrieve public key from private key file
variables["public_key"] = read_public_key(
    input(f"Enter the RSA private key path (or {default_key_path}): ")
    or default_key_path
)

In [ ]:
# Provide key name
variables["key_name"] = (
    input(f"Enter the key name (or {default_key_name}): ") or default_key_name
)

In [ ]:
# Provide AWS region
variables["region"] = (
    input(f"Enter the AWS region (or {default_region}): ") or default_region
)

# Terraform CLI

In [ ]:
# Creating options string for use in CLI commands
values = [f'"{key}={value}"' for key, value in variables.items()]
flags = ["-var"] * len(values)
options = " ".join([arg for tup in zip(flags, values) for arg in tup])
options

The `plan` subcommand is useful to sanity check what infrastructure terraform will modify in `apply`

In [ ]:
!terraform plan $options

`apply` will initiate the spot instance request. Afterwards, if the request is accepted by AWS, an EC2 instance will be accessible over SSH. For more on spot instances see [Spot Instances](https://docs.aws.amazon.com/AWSEC2/latest/UserGuide/using-spot-instances.html).

Once the EC2 instance is running it may connected to over SSH ([Connect to your Linux instance using SSH](https://docs.aws.amazon.com/AWSEC2/latest/UserGuide/AccessingInstancesLinux.html)):

```sh
ssh -i /path/my-key-pair.pem my-instance-user-name@my-instance-public-dns-name
```

In [ ]:
!terraform apply -auto-approve $options

Once finished with the EC2 instance it is important to run the `destroy` subcommand to cancel the spot instance request. Otherwise, the request may stay open and AWS will try and fulfill it.

In [ ]:
!terraform destroy -auto-approve $options